<a href="https://colab.research.google.com/github/AkshitAggarwal/TSAI_EVA5B2_Phase1/blob/main/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports all necessary libraries and methods from Pytorch. 

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    #Constructor function initializes an object of Net() and defines all the layers. 
    #Each layer is defined here separately and aren't connected to each other in any way yet. 
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #in_channel: 1; out_channel: 32; RF: 3 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #in_channel: 32; out_channel: 64; RF: 5 
        self.pool1 = nn.MaxPool2d(2, 2) #in_channel: 64; out_channel: 64; RF: 10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #in_channel: 64; out_channel: 128; RF: 12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #in_channel: 128; out_channel: 256; RF: 14
        self.pool2 = nn.MaxPool2d(2, 2) #in_channel: 256; out_channel: 256; RF: 28
        self.conv5 = nn.Conv2d(256, 512, 3) #in_channel: 256; out_channel: 512; RF: 30
        self.conv6 = nn.Conv2d(512, 1024, 3) #in_channel: 512; out_channel: 1024; RF: 32
        self.conv7 = nn.Conv2d(1024, 10, 3) #in_channel: 1024; out_channel: 10; RF: 34

    #Forward function takes an object and it passes through each layer sequentially. 
    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #Block 1: conv1 -> conv2 -> maxpool1
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #Block 2: conv3 -> conv4 -> maxpool2
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) #Block 3: conv5 -> conv6
        x = F.relu(self.conv7(x)) 
        x = x.view(-1, 10)
        return F.log_softmax(x)

Q1. The global receptive field of our network ends up being more than the size of the image that we input, how will that affect the results?

Q2. What does -1 in Output shape signify?

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() #Boolean function checks if cuda(GPU) is available or not.
device = torch.device("cuda" if use_cuda else "cpu") #Assigns cuda or cpu based on availability.
model = Net().to(device) #Converts our model into the respective device.
summary(model, input_size=(1, 28, 28)) #Prints the summary of our model based on an input size. 


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.6260, Accuracy: 4015/10000 (40%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.6092, Accuracy: 4064/10000 (41%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.6031, Accuracy: 4053/10000 (41%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.6064, Accuracy: 4064/10000 (41%)



loss=1.5402981042861938 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.54it/s]



Test set: Average loss: 1.6004, Accuracy: 4069/10000 (41%)

